# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [54]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [55]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,21.18,77,100,3.73,PN,1701330565
1,1,moose jaw,50.4001,-105.5344,-4.86,80,10,5.14,CA,1701330566
2,2,mahibadhoo,3.7833,72.9667,28.39,72,41,2.45,MV,1701330566
3,3,novodvinsk,64.4165,40.8122,-7.14,98,66,1.83,RU,1701330566
4,4,port-aux-francais,-49.3500,70.2167,2.47,84,64,20.08,TF,1701330439


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [61]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale=0.8,
    size = "Humidity",
    hover_cols=['City', 'Country', 'Humidity']

)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [57]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
# YOUR CODE HERE
print(len(ideal_city_data_df))
ideal_city_data_df = ideal_city_data_df.dropna(how="any")
print(len(ideal_city_data_df))

# Display sample data
ideal_city_data_df

63
62


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
46,46,thompson,55.7435,-97.8558,-21.91,71,0,4.63,CA,1701330437
53,53,port mathurin,-19.6833,63.4167,25.24,69,0,5.80,MU,1701330583
59,59,faya-largeau,17.9257,19.1043,25.94,21,0,8.23,TD,1701330586
87,87,shimoda,34.6667,138.9500,14.20,55,0,11.22,JP,1701330594
90,90,el granada,37.5027,-122.4694,12.54,89,0,0.00,US,1701330583
...,...,...,...,...,...,...,...,...,...,...
581,581,porbandar,21.6422,69.6093,27.91,35,0,3.45,IN,1701330783
582,582,dalandzadgad,43.5708,104.4250,-7.16,53,0,2.23,MN,1701330784
583,583,olavarria,-36.8927,-60.3225,14.27,88,0,3.00,AR,1701330784
585,585,california,38.3004,-76.5074,4.25,64,0,3.60,US,1701330785


### Step 3: Create a new DataFrame called `hotel_df`.

In [58]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = ideal_city_data_df.copy()
hotel_df = hotel_df[["City","Country",	"Lat",	"Lng",	"Humidity"	]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"] = ""

# Display sample data
# YOUR CODE HERE
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
46,thompson,CA,55.7435,-97.8558,71,
53,port mathurin,MU,-19.6833,63.4167,69,
59,faya-largeau,TD,17.9257,19.1043,21,
87,shimoda,JP,34.6667,138.9500,55,
90,el granada,US,37.5027,-122.4694,89,
...,...,...,...,...,...,...
581,porbandar,IN,21.6422,69.6093,35,
582,dalandzadgad,MN,43.5708,104.4250,53,
583,olavarria,AR,-36.8927,-60.3225,88,
585,california,US,38.3004,-76.5074,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [59]:
# Set parameters to search for a hotel
radius = 10000 
categories = "accommodation.hotel"

limit = 20
params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    longitude = row["Lng"]
    latitude = row["Lat"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"# YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
thompson - nearest hotel: Thompson Inn
port mathurin - nearest hotel: Escale Vacances
faya-largeau - nearest hotel: No hotel found
shimoda - nearest hotel: ホテルウラガ
el granada - nearest hotel: Beach House
san gabriel - nearest hotel: No hotel found
alaghsas - nearest hotel: Hôtel Zagado
mawlaik - nearest hotel: No hotel found
gulf shores - nearest hotel: Beachside Resort
mar del plata - nearest hotel: Gran hotel Mónaco
labrador city - nearest hotel: Carol Inn
port elizabeth - nearest hotel: Waterford Hotel
akrehamn - nearest hotel: Park Inn Haugesund Airport
as sulayyil - nearest hotel: No hotel found
kekri - nearest hotel: No hotel found
taft - nearest hotel: هتل شیرکوه تفت
i-n-salah - nearest hotel: باجودة
valparaiso - nearest hotel: Ibis Hotel
tura - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
geraldton - nearest hotel: Ocean Centre Hotel
vila velha - nearest hotel: Hotel Prainha
fort bragg - nearest hotel: Airborne Inn Lodging
comodor

,City,Country,Lat,Lng,Humidity,Hotel Name
46,thompson,CA,55.7435,-97.8558,71,Thompson Inn
53,port mathurin,MU,-19.6833,63.4167,69,Escale Vacances
59,faya-largeau,TD,17.9257,19.1043,21,No hotel found
87,shimoda,JP,34.6667,138.9500,55,ホテルウラガ
90,el granada,US,37.5027,-122.4694,89,Beach House
...,...,...,...,...,...,...
581,porbandar,IN,21.6422,69.6093,35,Toran Tourist Bungalow
582,dalandzadgad,MN,43.5708,104.4250,53,Хан Уул буудал
583,olavarria,AR,-36.8927,-60.3225,88,Santa Rosa Hotel
585,california,US,38.3004,-76.5074,64,Holiday Inn Express


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [62]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_1 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale=0.8,
    size = "Humidity",
    hover_cols=['City', 'Country', "Humidity", "Hotel Name"]
)

# Display the map
map_plot_1
# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)